### Now, we want to go over an example of wrapping an existing C software package
Software package we'll be working with: https://github.com/ntamas/plfit

This code fits power-law distributions to your data.

#### Making sure the C code works first
From the terminal:
```
$ git clone https://github.com/ntamas/plfit
$ mv plfit plfit_original_code
$ cd plfit_original_code/
$ mkdir build
$ cd build
$ cp ../../input_data.txt .
$ gcc -o plfit -I../src -lm `ls ../src/*.c | grep -F -v plgen.c`
$ ./plfit input_data.txt
$ cd ../../
```
The output will be print to the screen as follows:
```
input_data.txt:
	Discrete MLE
	alpha =      3.87363
	xmin  =      2.00000
	L     =    -89.79272
	D     =      0.01503
```

#### Writing the Julia wrapper
We will create a differnet folder for the wrapper for convenience:
From the terminal:
```
$ git clone https://github.com/ntamas/plfit
$ mv plfit plfit_Julia_wrapper
$ cd plfit_Julia_wrapper/
$ mkdir build
$ cd build
$ cp ../../input_data.txt .
$ cp ../../Makefile .
```

This requires changes to the file `../src/main.c`, move new `main.c` to folder `src`
```
$ rm ../src/main.c 
$ cp ../../main.c ../src/
```

Now we have all what we need, we need to compile and create the shared library
```
$ make
```
You should now see the file `libplfit.dylib` in you current folder

In [19]:
# remember that we are in the "4-wrappers_existing_packages" directory on this notebook still
const libpath = string(pwd(), "/plfit_Julia_wrapper/build/libplfit.dylib")
# tryin a quick test
ccall( (:process_file, libpath), # function + library
                        Void, # return type
                        (Ptr{UInt8},Ptr{UInt8},),
                        "input_data.txt","./plfit -c")

input_data.txt:
	Continuous MLE
	alpha =      4.75701
	xmin  =      6.00000
	L     =     -5.20292
	D     =      0.32735



In [26]:
# we can put this in a function and add some documentation
"""
plfit_julia_wrapper(filename,runoptions)
filename is the name of the file that contains your node degrees. Example file:
5
4
3
3
2
2
2
1
1
1

runoptions could be a combination of any of these options:
"    -h        shows this help message\n"
"    -v        shows version information\n"
"    -a RANGE  use legacy brute-force search for the optimal alpha\n"
"              when a discrete power-law distribution is fitted.\n"
"              RANGE must be in MIN:STEP:MAX format, the default\n"
"              is 1.5:0.01:3.5.\n"
"    -b        brief (but easily parseable) output format\n"
"    -c        force continuous fitting even when every sample\n"
"              is an integer\n"
"    -D VALUE  divide each sample in the input data by VALUE to prevent\n"
"              underflows when fitting discrete power-law distribution\n"
"    -e EPS    try to provide a p-value with a precision of EPS when\n"
"              the p-value is calculated using the exact method. The\n"
"              default is 0.01.\n"
"    -f        use finite-size correction\n"
"    -m XMIN   use XMIN as the minimum value for x instead of searching\n"
"              for the optimal value\n"
"    -M        print the first four central moments (i.e. mean, variance,\n"
"              skewness and kurtosis) of the input data to help\n"
"              assessing the shape of the pdf it may have come from.\n"
"    -p METHOD use METHOD to calculate the p-value. Must be one of\n"
"              skip, approximate or exact. Default is skip.\n"
"    -s SEED   use SEED to seed the random number generator\n"

example: plfit_julia_wrapper("input_data.txt","./plfit -c")

"""
function plfit_julia_wrapper(filename::ASCIIString,runoptions::ASCIIString)
    ccall( (:process_file, libpath), # function + library
                            Void, # return type
                            (Ptr{UInt8},Ptr{UInt8},),# input types
                            filename,runoptions)#input values
end

plfit_julia_wrapper (generic function with 1 method)

In [28]:
plfit_julia_wrapper("input_data.txt","./plfit -c") #run continuous

input_data.txt:
	Continuous MLE
	alpha =      4.75701
	xmin  =      6.00000
	L     =     -5.20292
	D     =      0.32735



In [29]:
plfit_julia_wrapper("input_data.txt","") #run default

input_data.txt:
	Discrete MLE
	alpha =      3.87363
	xmin  =      2.00000
	L     =    -89.79272
	D     =      0.01503



In [30]:
plfit_julia_wrapper("input_data.txt","./plfit -v") #get version

plfit 0.7


In [33]:
plfit_julia_wrapper("input_data.txt","./plfit -b -c") #brief and continuous

input_data.txt: C 4.75701 6 -5.20292 0.327351 nan


In [34]:
plfit_julia_wrapper("wrong_file.txt","./plfit -c") #pass a wrong file

wrong_file.txt: No such file or directory


In [37]:
plfit_julia_wrapper("input_data.txt","./plfit -Z") #invalid option

Invalid option `-Z'
